In [47]:
import requests
import pandas as pd
import json

# Replace these with your repository details
owner = 'lucamenotti'
repo = '5v5s'
branch = 'main'


api_url = f'https://api.github.com/repos/{owner}/{repo}/contents'

# # Make a request to the GitHub API
# # headers = {'Authorization': f'token {token}'}
# response = requests.get(url)
# if response.status_code == 200:
#     contents = response.json()
#     # Filter out the JSON files and get their raw URLs
#     json_urls = [file['download_url'] for file in contents if file['name'].endswith('.json')]
# else:
#     print(f"Failed to fetch files: {response.status_code}")
#     json_urls = []


def fetch_files_from_github(api_url, path=''):
    """Fetches JSON files from the given GitHub repository path."""
    files = []
    response = requests.get(f'{api_url}/{path}')
    if response.status_code == 200:
        for item in response.json():
            if item['type'] == 'file' and item['name'].endswith('.json'):
                files.append(item['download_url'])
            elif item['type'] == 'dir':
                files.extend(fetch_files_from_github(api_url, item['path']))
    else:
        print(f"Failed to fetch files: {response.status_code} - {response.text}")
    return files

def fetch_json_from_url(url):
    """Fetches JSON data from the given URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch JSON from {url}: {response.status_code} - {response.text}")
    return None

# Fetch JSON files from the GitHub repository
json_urls = fetch_files_from_github(api_url)


# Read the JSON files into DataFrames
dataframes = [pd.read_json(url) for url in json_urls]

print(json_urls)

# Combine the DataFrames into a single DataFrame (if necessary)
combined_df = pd.concat(dataframes, ignore_index=True)


['https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4146140154.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4507145109.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4523056888.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4523110972.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4523153945.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4526839461.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4526868306.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4576421436.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4576450279.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4595849876.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/main/json_files/4595879641.json', 'https://raw.githubusercontent.com/lucamenotti/5v5s/m

In [48]:
hold = []
for i in combined_df['participants']:
    hold.append(pd.Series(i))

df = combined_df.merge(pd.concat(hold, axis = 1).T, left_index= True, right_index=True,how = 'inner').drop(columns= ['participants', 'TOTAL_PLAYER_SCORE','TOTAL_SCORE_RANK','OBJECTIVE_PLAYER_SCORE','PUUID']).fillna(0)

In [49]:
player_aliases = {
    'EnderDragon7474': 'Ashalo1',
    'RaVe B00ST': 'Bryant',
    'CoomerHashira': 'Bryant',
    'Item Diversity': 'BloodStainedRose',
    'BigPeTe1239': 'DookieDemon12'
    # Add more mappings as needed
}
df['NAME'] = df['NAME'].apply(lambda x:player_aliases.get(x,x))

In [50]:
df

,matchId,gameDuration,gameVersion,ALL_IN_PINGS,ASSIST_ME_PINGS,ASSISTS,BAIT_PINGS,BARON_KILLS,BARRACKS_KILLED,BARRACKS_TAKEDOWNS,...,WARD_KILLED,WARD_PLACED,WARD_PLACED_DETECTOR,WAS_AFK,WAS_AFK_AFTER_FAILED_SURRENDER,WAS_EARLY_SURRENDER_ACCOMPLICE,WAS_LEAVER,WAS_SURRENDER_DUE_TO_AFK,WIN,COMBAT_PLAYER_SCORE
0,4146140154,1468845,11.24.414.4003,0,0,4,0,0,1,1,...,3,2,2,0,0,0,0,0,Win,0
1,4146140154,1468845,11.24.414.4003,0,0,4,0,0,0,0,...,2,11,0,0,0,0,0,0,Win,0
2,4146140154,1468845,11.24.414.4003,0,0,13,0,0,0,1,...,4,17,2,0,0,0,0,0,Win,0
3,4146140154,1468845,11.24.414.4003,0,0,6,0,0,0,0,...,2,9,3,0,0,0,0,0,Win,0
4,4146140154,1468845,11.24.414.4003,0,0,7,0,0,0,1,...,1,4,2,0,0,0,0,0,Win,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,5031483856,1970770,14.13.598.9749,0,0,3,0,0,0,0,...,4,9,0,0,0,0,0,0,Fail,0
596,5031483856,1970770,14.13.598.9749,0,1,7,0,0,0,0,...,2,12,1,0,0,0,0,0,Fail,0
597,5031483856,1970770,14.13.598.9749,0,0,10,0,0,0,0,...,10,26,2,0,0,0,0,0,Fail,0
598,5031483856,1970770,14.13.598.9749,0,1,8,0,0,0,0,...,1,12,1,0,0,0,0,0,Fail,0


In [51]:
multi_type = []
for col in df.columns:
    if df[col].apply(type).nunique() != 1:
        multi_type.append(col)
multi_type

['ALL_IN_PINGS',
 'ASSIST_ME_PINGS',
 'BAIT_PINGS',
 'BARRACKS_TAKEDOWNS',
 'BASIC_PINGS',
 'COMMAND_PINGS',
 'DANGER_PINGS',
 'ENEMY_MISSING_PINGS',
 'ENEMY_VISION_PINGS',
 'GET_BACK_PINGS',
 'HOLD_PINGS',
 'HORDE_KILLS',
 'HQ_TAKEDOWNS',
 'ID',
 'LARGEST_ABILITY_DAMAGE',
 'LARGEST_ATTACK_DAMAGE',
 'LAST_TAKEDOWN_TIME',
 'Missions_ChampionsKilled',
 'Missions_CreepScore',
 'Missions_GoldFromStructuresDestroyed',
 'Missions_GoldFromTurretPlatesTaken',
 'Missions_HealingFromLevelObjects',
 'Missions_MinionsKilled',
 'Missions_TurretPlatesDestroyed',
 'NEED_VISION_PINGS',
 'ON_MY_WAY_PINGS',
 'PLAYER_AUGMENT_1',
 'PLAYER_AUGMENT_2',
 'PLAYER_AUGMENT_3',
 'PLAYER_AUGMENT_4',
 'PLAYER_SUBTEAM',
 'PLAYER_SUBTEAM_PLACEMENT',
 'PUSH_PINGS',
 'RETREAT_PINGS',
 'RIFT_HERALD_KILLS',
 'TOTAL_TIME_CROWD_CONTROL_DEALT_TO_CHAMPIONS',
 'TURRET_TAKEDOWNS',
 'VISION_CLEARED_PINGS',
 'WAS_LEAVER',
 'WAS_SURRENDER_DUE_TO_AFK']

In [52]:
for i in df.columns:
    df[i] = pd.to_numeric(df[i], errors = 'ignore')


In [53]:
multi_type = []
for col in df.columns:
    if df[col].apply(type).nunique() != 1:
        multi_type.append(col)
multi_type

[]

In [54]:
df.groupby('NAME').sum()['CHAMPIONS_KILLED'].sort_values(ascending = False)

NAME
adostraa            411
TylerToTheT212      404
IbelieveMeYou       356
nigeriacoc          297
Ashalo1             295
Dyneliex            144
DonutDude17         127
GasterBat           124
ihyash              112
Bryant               97
Little Bro Eman      74
Babar                74
MilkLL               63
dizzycat11           58
Jickens              55
DookieDemon12        51
BloodStainedRose     48
uWumnati             45
batlife10            31
p1eissoawesome       29
somethin idk         15
MonkeyMan4912        11
trunksthefuture      11
Su5hiS                8
Rin Reforged          8
TinyTiger             7
takoyakiii            7
aeti                  6
Mossidy               5
kebinlee              5
Detraplex             1
Name: CHAMPIONS_KILLED, dtype: int64

In [55]:
temp = df.groupby('matchId').sum()[['NAME']]
temp[temp['NAME'].str.contains('adostraa') != True].index

df[(df['matchId'] == 4939492680) | (df['matchId'] == 4939565193)]['NAME']

480    trunksthefuture
481      IbelieveMeYou
482             Bryant
483          batlife10
484      DookieDemon12
485            Ashalo1
486             ihyash
487        DonutDude17
488          Detraplex
489         nigeriacoc
490    trunksthefuture
491           uWumnati
492     TylerToTheT212
493             ihyash
494             Bryant
495            Ashalo1
496             Su5hiS
497        DonutDude17
498      IbelieveMeYou
499         nigeriacoc
Name: NAME, dtype: object

In [56]:
df.groupby('NAME').count()['CHAMPIONS_KILLED'].sort_values(ascending = False)

NAME
nigeriacoc          59
adostraa            58
TylerToTheT212      54
IbelieveMeYou       52
Ashalo1             46
ihyash              38
DonutDude17         36
GasterBat           31
Bryant              29
batlife10           25
uWumnati            25
Dyneliex            23
DookieDemon12       20
Babar               19
MilkLL              13
Little Bro Eman     10
Jickens              9
BloodStainedRose     8
somethin idk         8
dizzycat11           7
p1eissoawesome       5
MonkeyMan4912        4
Su5hiS               3
Mossidy              3
aeti                 3
takoyakiii           3
TinyTiger            2
kebinlee             2
trunksthefuture      2
Rin Reforged         2
Detraplex            1
Name: CHAMPIONS_KILLED, dtype: int64

In [57]:
def standardize (series):
    return (series - series.mean())/series.std()

temp_df = df
standardize(temp_df[['NAME','TOTAL_DAMAGE_DEALT']].groupby('NAME').mean()).sort_values('TOTAL_DAMAGE_DEALT', ascending = False)


,TOTAL_DAMAGE_DEALT
NAME,
trunksthefuture,2.884448
adostraa,1.305667
Little Bro Eman,1.292901
dizzycat11,1.155907
Jickens,0.842338
Dyneliex,0.714659
IbelieveMeYou,0.666431
Ashalo1,0.658003
MilkLL,0.561019


In [58]:
for col in df.columns:
    print(col)

matchId
gameDuration
gameVersion
ALL_IN_PINGS
ASSIST_ME_PINGS
ASSISTS
BAIT_PINGS
BARON_KILLS
BARRACKS_KILLED
BARRACKS_TAKEDOWNS
BASIC_PINGS
BOUNTY_LEVEL
CHAMPION_MISSION_STAT_0
CHAMPION_MISSION_STAT_1
CHAMPION_MISSION_STAT_2
CHAMPION_MISSION_STAT_3
CHAMPIONS_KILLED
CHAMPION_TRANSFORM
COMMAND_PINGS
CONSUMABLES_PURCHASED
DANGER_PINGS
DOUBLE_KILLS
DRAGON_KILLS
ENEMY_MISSING_PINGS
ENEMY_VISION_PINGS
EXP
FRIENDLY_DAMPEN_LOST
FRIENDLY_HQ_LOST
FRIENDLY_TURRET_LOST
GAME_ENDED_IN_EARLY_SURRENDER
GAME_ENDED_IN_SURRENDER
GET_BACK_PINGS
GOLD_EARNED
GOLD_SPENT
HOLD_PINGS
HORDE_KILLS
HQ_KILLED
HQ_TAKEDOWNS
ID
INDIVIDUAL_POSITION
ITEM0
ITEM1
ITEM2
ITEM3
ITEM4
ITEM5
ITEM6
ITEMS_PURCHASED
KEYSTONE_ID
KILLING_SPREES
LARGEST_ABILITY_DAMAGE
LARGEST_ATTACK_DAMAGE
LARGEST_CRITICAL_STRIKE
LARGEST_KILLING_SPREE
LARGEST_MULTI_KILL
LAST_TAKEDOWN_TIME
LEVEL
LONGEST_TIME_SPENT_LIVING
MAGIC_DAMAGE_DEALT_PLAYER
MAGIC_DAMAGE_DEALT_TO_CHAMPIONS
MAGIC_DAMAGE_TAKEN
MINIONS_KILLED
Missions_ChampionsKilled
Missions_Creep